In [1]:
from utils import *
import pymongo
import pandas as pd
import json

In [2]:
config = load_config()
mongo_client = pymongo.MongoClient(get_mongo_url(config))
db = mongo_client['ChameleonSimulator']

In [3]:
db.collection_names()

/tmp/ipykernel_59551/1579878703.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  db.collection_names()


['osg_jobs', 'resource_pool']

In [4]:
rsrc_pool = db['resource_pool']

In [16]:
result = rsrc_pool.find({"$and":[
                {"HOST_NAME (PHYSICAL)": "fa9dc77d7da1a9517f2a7fc1b5141824"}, 
                {"pool": "osg"},
                {"backfill": {"$elemMatch": {"GlobalJobId": "login05_osgconnect_net#11290350_6016#1624697819"}}}]})

result = pd.DataFrame(result)

In [17]:
result

,_id,EVENT_TIME,EVENT_TIME_SEC,HOST_NAME (PHYSICAL),EVENT,PROPERTIES,node_type,status,pool,cpus,inuse_cpus,memory,inuse_memory,backfill,ready_for_osg
0,60fcbdb8e6f483a94144096a,2018-03-14 11:40:04,86528404.0,fa9dc77d7da1a9517f2a7fc1b5141824,UPDATE,"{'architecture.smt_size': '48', 'node_type': '...",compute_haswell,inuse,osg,2,2,131072,6656,[{'GlobalJobId': 'login05_osgconnect_net#11290...,1.627177e+09


In [ ]:
result = rsrc_pool.aggregate([
    {"$match": {"pool": "osg"}},
    {"$project": {
        "Machine": "$HOST_NAME (PHYSICAL)",
        "free_cpus": {"$subtract": ["$cpus", "$inuse_cpus"]}, 
        "free_memory": {"$subtract": ["$memory", "$inuse_memory"]}
    }},
    {"$match": { 
        "$and": [
            {"free_cpus": {"$gte": 1}},
            {"free_memory": {"$gte": 1}}
        ]}
    },
    {"$limit": 1}
])

result = pd.DataFrame(list(result))
print(result)
result.apply(lambda row: str(row['_id']), axis=1)
result = result.to_dict(orient='records')
# json.dumps(result)
# result['_id'] = [str(x) for x in result['_id'].to_list()]
# result = result.to_dict(orient="records")
# (result)
# result = rsrc_pool.update_one(
#     {'_id': result[0]['_id']}, 
#     {"$inc": {
#         "inuse_cpus": 8, 
#         "inuse_memory": 8
#         },
#         "$pull": {"backfill": {"GlobalJobId": "abc"}}
#     }
# )
# print(result)

In [ ]:
result = rsrc_pool.aggregate([
    {"$match": {"$and": [ {"node_type": 'compute_haswell'}, {"pool": "chameleon"}, {"status": "inuse"} ] }},
    {"$limit": 10}
])
result = list(result)
print(result)